# Applied Data Analysis Project
**Team**: ToeStewBrr - Alexander Sternfeld, Marguerite Thery, Antoine Bonnet, Hugo Bordereaux

**Dataset**: CMU Movie Summary Corpus


In [ ]:
import requests
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
from nltk.tree import Tree
import xml.etree.ElementTree as ET
import itertools

from load_data import *


## 1. Loading data

We first extract all files from the [MoviesSummaries dataset](http://www.cs.cmu.edu/~ark/personas/). 

`corenlp_plot_summaries.tar.gz [628 M, separate download]`: The plot summary of each movie, run through the Stanford CoreNLP pipeline (tagging, parsing, NER and coref). Each filename begins with the Wikipedia movie ID (which indexes into movie.metadata.tsv).

We extract all coreNLP files, then uncompress them to the XML format. 

Note: Extraction of CoreNLP files takes 15 minutes, while conversion takes 30 seconds. 

In [ ]:
download_data()

## 2. Pre-processing data

### 2.1. Plot summaries

`plot_summaries.txt [29 M]`: Plot summaries of 42,306 movies extracted from the November 2, 2012 dump of English-language Wikipedia.  Each line contains the Wikipedia movie ID (which indexes into movie.metadata.tsv) followed by the summary.

In [ ]:
plot_df = load_plot_df()
plot_df

In [ ]:
# For Hugo: this method stems the words to their lexical root. 
# Implement Stemming using out of the box Porter algorithm
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
plot_stemmed = [[stemmer.stem(word) for word in sentence.split(" ")] for sentence in plot_df.iloc[:5].Summary]
plot_stemmed = [" ".join(sentence) for sentence in plot_stemmed]


### 2.2. Movie metadata

`movie.metadata.tsv.gz [3.4 M]`: Metadata for 81,741 movies, extracted from the Noverber 4, 2012 dump of Freebase.  Tab-separated; columns:

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie name
4. Movie release date
5. Movie box office revenue
6. Movie runtime
7. Movie languages (Freebase ID:name tuples)
8. Movie countries (Freebase ID:name tuples)
9. Movie genres (Freebase ID:name tuples)


In [ ]:
movie_df = load_movie_df()
movie_df

### 2.3. Character metadata

`character.metadata.tsv.gz [14 M]`: Metadata for 450,669 characters aligned to the movies above, extracted from the November 4, 2012 dump of Freebase.  Tab-separated; columns:

1. Wikipedia movie ID
2. Freebase movie ID
3. Movie release date
4. Character name
5. Actor date of birth
6. Actor gender
7. Actor height (in meters)
8. Actor ethnicity (Freebase ID)
9. Actor name
10. Actor age at movie release
11. Freebase character/actor map ID
12. Freebase character ID
13. Freebase actor ID


In [ ]:
char_df = load_char_df()
char_df

### 2.4. Name clusters

`name.clusters.txt`: 970 unique character names used in at least two different movies, along with 2,666 instances of those types.  The ID field indexes into the Freebase character/actor map ID in character.metadata.tsv.

In [ ]:
names_df = load_names_df()
names_df

### 2.5. TV Tropes Clusters

`tvtropes.clusters.txt`: 72 character types drawn from tvtropes.com, along with 501 instances of those types.  The ID field indexes into the Freebase character/actor map ID in character.metadata.tsv.

In [ ]:
cluster_df = load_cluster_df()
cluster_df


We now join the TV tropes clusters with movie.metadata so we are able to access movie genre and filter on romance. 

In [ ]:
# TODO: Fix this
cluster_char = cluster_df.merge(char_df, on='Freebase character/map ID')
cluster_char_movie = cluster_char.merge(movie_df, on='Freebase ID')
romance_cluster = cluster_char_movie[cluster_char_movie['Genres'].apply(lambda x: 'Roman' in x)]
romance_cluster.groupby(romance_cluster['Cluster']).size().sort_values(ascending=False)
romance_cluster

### 2.6. Missing values and outliers

#### Identifying missing values

In [ ]:
# Missing values
movie_cols = ['Wikipedia ID', 'Freebase ID', 'Name', 'Release date', 
'Box office revenue', 'Runtime', 'Languages', 'Countries', 'Genres']
for column in movie_cols[3:]:
    number_missing = movie_df[column].isna().sum()
    print('Missing values of ', column, ': ', number_missing, ' (',round(100*number_missing/movie_df.shape[0],2), '%)')


#### Outliers
- We note that there is one movie with a release date in the year 1010, we manually correct this to the date 2010
- There are many movies with very short runtimes (under 5 minutes). We manually verified a sample of these movies, and have found that the runtimes are correct.
- There are over a hundred movies with runtimes over 5 hours. We have found that most of these data points correspond to series, where the runtime is the sum of the episode lengths.
- One movie has a runtime of over a million minutes ('Zero tolerance'). This lengths is manually corrected to 88 minutes.

In [ ]:
# Isolate the year from the release date
getYear = lambda x: x[slice(0,4)] if type(x) == str else x
movie_df['Release date'] = movie_df['Release date'].apply(getYear)

# Identify suspicious movie with release date 1010, manually correct
outlier_release_date = movie_df[movie_df['Release date'].apply(lambda x: int(x.split('-')[0]) < 1850 if type(x)==str else False)]
movie_df.at[29666067, 'Release date'] = 2010

# Identify suspicious movies with long runtimes
movie_df[movie_df['Runtime'] > 300]
movie_df.at[10815585, 'Runtime'] = 88

## 3. Exploratory Data Analysis


### 3.1. Analysing romantic genres

One notices that there are several types of romantic movies: romantic comedy, romance film, romantic drama. 

In [ ]:
romance_genres = ['Romantic comedy', 'Romance Film', 'Romantic drama', 'Romantic fantasy', 'Romantic thriller']
is_romantic = lambda i: lambda x: any(y in romance_genres[i] for y in x) if type(x) == list else False
is_not_romantic = lambda i: lambda x: not any(y in romance_genres[i] for y in x) if type(x) == list else False
romance_movies = movie_df[movie_df['Genres'].apply(is_romantic(slice(0, 5)))]
non_romance_movies = movie_df[movie_df['Genres'].apply(is_not_romantic(slice(0, 5)))]

In [ ]:
#Organize by category
romantic_comedy = romance_movies.loc[movie_df['Genres'].apply(is_romantic(0))]
romantic_film = romance_movies.loc[movie_df['Genres'].apply(is_romantic(1))]
romantic_drama = romance_movies.loc[movie_df['Genres'].apply(is_romantic(2))]
romantic_fantasy = romance_movies.loc[movie_df['Genres'].apply(is_romantic(3))]
romantic_thriller = romance_movies.loc[movie_df['Genres'].apply(is_romantic(4))]
print('Roman' , romance_movies.shape[0])
print('Romantic comedies: ', romantic_comedy.shape[0], '\nRomantic films: ', romantic_film.shape[0], '\nRomantic drama: ', romantic_drama.shape[0], '\nRomantic fantasy: ', romantic_fantasy.shape[0], '\nRomantic thriller: ', romantic_thriller.shape[0])
print('Total number of films: ', movie_df.shape[0])

### 3.2. Romantic movies runtime

In [ ]:
ax = sns.kdeplot(romantic_comedy['Runtime'], color='blue')
ax = sns.kdeplot(romantic_drama['Runtime'], color='green')
ax = sns.kdeplot(romantic_film['Runtime'], color='red')
ax = sns.kdeplot(romantic_fantasy['Runtime'], color='orange')
ax = sns.kdeplot(non_romance_movies['Runtime'], color = 'purple')
ax.set_xlim(0, 200)
ax.legend(['Romantic comedy', 'Romantic drama', 'Romance Film', 'Romantic fantasy', 'Non romantic movies'])


### 3.3. Romantic movies box office revenue

In [ ]:
#Does not give a good view
#combined_box_office = pd.DataFrame({'Romantic comedy': romantic_comedy['Box office revenue'], 'Romance Film': romantic_film['Box office revenue'], 'Romantic drama': romantic_drama['Box office revenue'], 'Romantic fantasy': romantic_fantasy['Box office revenue']})
#sns.boxplot(combined_box_office)

In [ ]:
ax = sns.kdeplot(romantic_comedy['Box office revenue'], log_scale=True, color='blue')
ax = sns.kdeplot(romantic_drama['Box office revenue'], log_scale=True, color='green')
ax = sns.kdeplot(romantic_film['Box office revenue'], log_scale=True, color='red')
ax = sns.kdeplot(romantic_fantasy['Box office revenue'], log_scale=True, color='orange')
ax = sns.kdeplot(non_romance_movies['Box office revenue'], log_scale=True, color='purple')
ax.legend(['Romantic comedy', 'Romantic drama', 'Romance Film', 'Romantic fantasy', 'Non romantic movies'])


### 3.4. Romantic movies countries

In [ ]:
get_countries = lambda x: len(x) if type(x) == list else np.nan
romantic_comedy['number_countries'] = romantic_comedy['Countries'].apply(get_countries)
romantic_fantasy['number_countries'] = romantic_fantasy['Countries'].apply(get_countries)
romantic_film['number_countries'] = romantic_film['Countries'].apply(get_countries)
romantic_drama['number_countries'] = romantic_drama['Countries'].apply(get_countries)

combined_numb_countries = pd.DataFrame({
    'Romantic comedy': romantic_comedy['number_countries'], 
    'Romance Film': romantic_film['number_countries'], 
    'Romantic drama': romantic_drama['number_countries'], 
    'Romantic fantasy': romantic_fantasy['number_countries']})

print('Percentage romantic comedy movie countries > 1: ', round(romantic_comedy[romantic_comedy['number_countries']> 1].shape[0]/romantic_comedy.shape[0], 2), '%')
print('Other countries can be added in code...')

### 3.5. Movie languages

In [ ]:
#Get languages whole movie set
movies_language = movie_df[movie_df['Languages'].notnull()]
languages=movies_language['Languages'].sum()
values, counts = np.unique(languages, return_counts=True)
print('5 most common languages in movies are: ')
print(values[counts.argsort()[-5:][::-1]])

#Get languages romantic movies overall
romance_movies_lang = romance_movies[romance_movies['Languages'].notnull()]
languages_romance = romance_movies_lang.Languages.sum()
values, counts = np.unique(languages_romance, return_counts=True)
print('\n5 most common languages in romantic movies: ')
print(values[counts.argsort()[-5:][::-1]])


rom_com_known = romantic_comedy[romantic_comedy['Languages'].notnull()]
languages_romcom = rom_com_known.Languages.sum()
values, counts = np.unique(languages_romcom, return_counts=True)
print('\n5 most common languages in romantic comedies: ')
print(values[counts.argsort()[-5:][::-1]])

### 3.6. Evolution over time

#### 3.6.1 Box office revenue over time

In [ ]:
year_box_office = non_romance_movies[['Release date', 'Box office revenue']].dropna()
#year_box_office = movie_df[['Release date', 'Box office revenue']].dropna()
year_box_office_romance = romance_movies[['Release date', 'Box office revenue']].dropna()
yearly_revenue_total = year_box_office.groupby('Release date').mean()
yearly_revenue_romance = year_box_office_romance.groupby('Release date').mean()
plt.plot(yearly_revenue_total.index, yearly_revenue_total['Box office revenue'],'b-',  label = 'Average box office revenue')
plt.fill_between(yearly_revenue_total.index, yearly_revenue_total['Box office revenue'] - yearly_revenue_total['Box office revenue'].std(), yearly_revenue_total['Box office revenue'] + yearly_revenue_total['Box office revenue'].std(), color='blue', alpha=0.2)
plt.plot(yearly_revenue_romance.index, yearly_revenue_romance['Box office revenue'],'r-',  label = 'Average box office revenue romantic films')
plt.fill_between(yearly_revenue_romance.index, yearly_revenue_romance['Box office revenue'] - yearly_revenue_romance['Box office revenue'].std(), yearly_revenue_romance['Box office revenue'] + yearly_revenue_romance['Box office revenue'].std(), color='red', alpha=0.2)
plt.legend()
ax = plt.gca()
plt.ylabel('Box office revenue')
plt.xlabel('Year')
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
plt.show()

### 3.6.2. Evolution of runtime

In [ ]:
yearly_runtime_non_romance = non_romance_movies[['Release date', 'Runtime']].dropna().groupby('Release date').mean()
yearly_runtime_romance = romance_movies[['Release date', 'Runtime']].dropna().groupby('Release date').mean()
plt.plot(yearly_runtime_non_romance.index, yearly_runtime_non_romance['Runtime'], 'b-',  label = 'Average runtime non romantic movies')
plt.fill_between(yearly_runtime_non_romance.index, yearly_runtime_non_romance['Runtime'] - yearly_runtime_non_romance['Runtime'].std(), yearly_runtime_non_romance['Runtime'] + yearly_runtime_non_romance['Runtime'].std(), color='blue', alpha=0.2)
plt.plot(yearly_runtime_romance.index, yearly_runtime_romance['Runtime'], 'r-',  label = 'Average runtime romantic movies')
plt.fill_between(yearly_runtime_romance.index, yearly_runtime_romance['Runtime'] - yearly_runtime_romance['Runtime'].std(), yearly_runtime_romance['Runtime'] + yearly_runtime_romance['Runtime'].std(), color='red', alpha=0.2)
plt.legend()
ax = plt.gca()
plt.ylabel('Runtime')
plt.xlabel('Year')
ax.xaxis.set_major_locator(plt.MaxNLocator(10))
plt.show()